In [31]:
import pandas as pd
import numpy as np
from sklearn import ensemble, preprocessing

In [32]:
#Load dataset
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
spray=pd.read_csv('spray.csv')
sample = pd.read_csv('sampleSubmission.csv')
weather=pd.read_csv('weather.csv')
weather.head()
weather['Station'].value_counts()
weather.columns

Index(['Station', 'Date', 'Tmax', 'Tmin', 'Tavg', 'Depart', 'DewPoint',
       'WetBulb', 'Heat', 'Cool', 'Sunrise', 'Sunset', 'CodeSum', 'Depth',
       'Water1', 'SnowFall', 'PrecipTotal', 'StnPressure', 'SeaLevel',
       'ResultSpeed', 'ResultDir', 'AvgSpeed'],
      dtype='object')

In [33]:
#Get Labels
labels=train.WnvPresent.values
print(labels)


[0 0 0 ... 0 0 0]


In [34]:
#There are many missing values in CodeSum of weather, so drop it
weather= weather.drop('CodeSum',axis=1)


In [35]:
weather_stn1 = weather[weather['Station']==1]
weather_stn2 = weather[weather['Station']==2]
weather_stn1 = weather_stn1.drop('Station', axis=1)
weather_stn2 = weather_stn2.drop('Station', axis=1)
print(weather_stn1.columns)
print(weather_stn2.columns)
weather = pd.merge(weather_stn1,weather_stn2, on='Date')
weather.columns

Index(['Date', 'Tmax', 'Tmin', 'Tavg', 'Depart', 'DewPoint', 'WetBulb', 'Heat',
       'Cool', 'Sunrise', 'Sunset', 'Depth', 'Water1', 'SnowFall',
       'PrecipTotal', 'StnPressure', 'SeaLevel', 'ResultSpeed', 'ResultDir',
       'AvgSpeed'],
      dtype='object')
Index(['Date', 'Tmax', 'Tmin', 'Tavg', 'Depart', 'DewPoint', 'WetBulb', 'Heat',
       'Cool', 'Sunrise', 'Sunset', 'Depth', 'Water1', 'SnowFall',
       'PrecipTotal', 'StnPressure', 'SeaLevel', 'ResultSpeed', 'ResultDir',
       'AvgSpeed'],
      dtype='object')


Index(['Date', 'Tmax_x', 'Tmin_x', 'Tavg_x', 'Depart_x', 'DewPoint_x',
       'WetBulb_x', 'Heat_x', 'Cool_x', 'Sunrise_x', 'Sunset_x', 'Depth_x',
       'Water1_x', 'SnowFall_x', 'PrecipTotal_x', 'StnPressure_x',
       'SeaLevel_x', 'ResultSpeed_x', 'ResultDir_x', 'AvgSpeed_x', 'Tmax_y',
       'Tmin_y', 'Tavg_y', 'Depart_y', 'DewPoint_y', 'WetBulb_y', 'Heat_y',
       'Cool_y', 'Sunrise_y', 'Sunset_y', 'Depth_y', 'Water1_y', 'SnowFall_y',
       'PrecipTotal_y', 'StnPressure_y', 'SeaLevel_y', 'ResultSpeed_y',
       'ResultDir_y', 'AvgSpeed_y'],
      dtype='object')

In [36]:
weather = weather.replace('M', -1)
weather = weather.replace('-', -1)
weather = weather.replace('T', -1)
weather = weather.replace(' T', -1)
weather = weather.replace('  T', -1)


In [37]:
train.head(1)


,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,0


In [38]:
test.head(1)

,Id,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy
0,1,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9


In [39]:
def create_month(x):
    return x.split('-')[1]

def create_day(x):
    return x.split('-')[2]

train['month'] = train.Date.apply(create_month)
train['day'] = train.Date.apply(create_day)
test['month'] = test.Date.apply(create_month)
test['day'] = test.Date.apply(create_day)

In [40]:
train['Lat_int'] = train.Latitude.apply(int)
train['Long_int']= train.Longitude.apply(int)
test['Lat_int']  = test.Latitude.apply(int)
test['Long_int'] = test.Longitude.apply(int)

In [41]:
train=train.drop(['Address','AddressNumberAndStreet','NumMosquitos','WnvPresent'],axis=1)
test=test.drop(['Address','AddressNumberAndStreet','Id',],axis=1)


In [42]:
#Merge with weather data
train = train.merge(weather, on='Date')
test = test.merge(weather, on='Date')
train=train.drop('Date',axis=1)
test=test.drop('Date',axis=1)


In [43]:
#convert categorical value in species to numerical value, there are 7 classes
train['Species'].unique()

array(['CULEX PIPIENS/RESTUANS', 'CULEX RESTUANS', 'CULEX PIPIENS',
       'CULEX SALINARIUS', 'CULEX TERRITANS', 'CULEX TARSALIS',
       'CULEX ERRATICUS'], dtype=object)

In [44]:
lbl = preprocessing.LabelEncoder()
train['Species']=lbl.fit_transform(train['Species'])
test['Species']=lbl.fit_transform(test['Species'])
train['Street']=lbl.fit_transform(train['Street'])
test['Street']=lbl.fit_transform(test['Street'])
train['Trap']=lbl.fit_transform(train['Trap'])
test['Trap']=lbl.fit_transform(test['Trap'])

In [45]:
train = train.loc[:,(train != -1).any(axis=0)]
test = test.loc[:,(test != -1).any(axis=0)]


In [46]:
train.head(1)

,Species,Block,Street,Trap,Latitude,Longitude,AddressAccuracy,month,day,Lat_int,...,DewPoint_y,WetBulb_y,Heat_y,Cool_y,PrecipTotal_y,StnPressure_y,SeaLevel_y,ResultSpeed_y,ResultDir_y,AvgSpeed_y
0,2,41,32,1,41.95469,-87.800991,9,05,29,41,...,59,66,0,12,0.00,29.44,30.09,5.8,16,7.4


In [47]:
clf = ensemble.RandomForestClassifier(n_jobs=-1, n_estimators=1000, min_samples_split=2)
clf.fit(train, labels)

RandomForestClassifier(n_estimators=1000, n_jobs=-1)

In [48]:
predictions = clf.predict_proba(test)[:,1]
clf.predict_proba(test)

array([[0.98383164, 0.01616836],
       [0.99386667, 0.00613333],
       [0.98207381, 0.01792619],
       ...,
       [0.99077308, 0.00922692],
       [0.99077308, 0.00922692],
       [0.98542328, 0.01457672]])

In [53]:
sample['WnvPresent'] = predictions
sample.loc[(sample['WnvPresent']>0.5)]

,Id,WnvPresent
107709,107710,0.519083
107710,107711,0.519083
108090,108091,0.560482
108092,108093,0.568413
108097,108098,0.567413


In [ ]:
sample.to_csv('output.csv', index=False)

In [ ]:
print("predictions above 0.5 = ", sum(predictions > 0.5))